In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210711.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,06/07/2021 17:38:31,Exotica (1994),La Haine (1995),Man Bites Dog (1992),Airborne (1993),OOPS ALL DONALD SUTHERLAND,THE FUCKIN' POLICE,PREVIOUSLY SUBMITTED,DISGUISES,[Default to Nicolas Cage]
1,06/07/2021 18:04:19,Airborne (1993),La Haine (1995),NaN,NaN,PREVIOUSLY SUBMITTED,DISGUISES,THE FUCKIN' POLICE,OOPS ALL DONALD SUTHERLAND,[Default to Nicolas Cage]
2,06/07/2021 18:08:27,Man Bites Dog (1992),Exotica (1994),La Haine (1995),Airborne (1993),PREVIOUSLY SUBMITTED,THE FUCKIN' POLICE,OOPS ALL DONALD SUTHERLAND,DISGUISES,[Default to Nicolas Cage]
3,07/07/2021 14:12:56,La Haine (1995),Airborne (1993),Exotica (1994),Man Bites Dog (1992),PREVIOUSLY SUBMITTED,THE FUCKIN' POLICE,DISGUISES,OOPS ALL DONALD SUTHERLAND,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Exotica (1994)', 'La Haine (1995)', 'Man Bites Dog (1992)',
       'Airborne (1993)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Exotica (1994)': <Candidate('Exotica (1994)')>,
 'La Haine (1995)': <Candidate('La Haine (1995)')>,
 'Man Bites Dog (1992)': <Candidate('Man Bites Dog (1992)')>,
 'Airborne (1993)': <Candidate('Airborne (1993)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Exotica (1994), La Haine (1995), Man Bites Dog (1992), Airborne (1993))>,
 <Ballot(Airborne (1993), La Haine (1995))>,
 <Ballot(Man Bites Dog (1992), Exotica (1994), La Haine (1995), Airborne (1993))>,
 <Ballot(La Haine (1995), Airborne (1993), Exotica (1994), Man Bites Dog (1992))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate               Votes  Status
--------------------  -------  --------
La Haine (1995)             1  Hopeful
Exotica (1994)              1  Hopeful
Airborne (1993)             1  Hopeful
Man Bites Dog (1992)        1  Rejected

FINAL RESULT
Candidate               Votes  Status
--------------------  -------  --------
Exotica (1994)              2  Elected
La Haine (1995)             1  Rejected
Airborne (1993)             1  Rejected
Man Bites Dog (1992)        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

FINAL RESULT
Candidate                     Votes  Status
--------------------------  -------  --------
PREVIOUSLY SUBMITTED              3  Elected
OOPS ALL DONALD SUTHERLAND        1  Rejected
THE FUCKIN' POLICE                0  Rejected
DISGUISES                         0  Rejected
[Default to Nicolas Cage]         0  Rejected

